In [34]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
# from sklearn.cluster import SpectralClustering
# from sklearn import metrics
# from gensim.models import KeyedVectors
# import os
import re


import italian_dictionary
import gensim.downloader
from nltk.corpus import wordnet as wn
# from nltk.stem import SnowballStemmer
# import simplemma
# nltk.download('wordnet')
# nltk.download('omw')
# import spacy

from Levenshtein import distance as lev
import math
from collections import Counter

In [35]:
PATH_INPUT = r"C:\Users\Nauel\Desktop\Lavoro UNI\data\input"
PATH_OUTPUT = r"C:\Users\Nauel\Desktop\Lavoro UNI\data\output"
PATH_CHECKPOINTS = r"C:\Users\Nauel\Desktop\Lavoro UNI\Checkpoints"

In [36]:
# # IMPORT DEL TRANSFORMER DI RICONOSCIMENTO LINGUA (MULTILINGUA)
# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [37]:
# IMPORT DEL TRANSFORMER DI RICONOSCIMENTO LINGUA (INGLESE)
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

c:\Users\Nauel\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


In [38]:
# LETTURA FILE DI INPUT E ISOLAZIONE COLONNA DI INTERESSE
df_samples = pd.read_stata(f'{PATH_INPUT}\score_treccani2.dta')
df_samples.head()

,parola,parola2,w,d0,s0,no_match
0,A CALCIO,A CALCIO,a calcio,,,0.0
1,A PALLAVOLO,A PALLAVOLO,a pallavolo,,,0.0
2,A SCUOLA,A SCUOLA,a scuola,,,0.0
3,ABBINARE I COLORI,ABBINARE I COLORI,abbinare i colori,,,0.0
4,ACCOGLIENTE,ACCOGLIENTE,accogliente,"accogliènte agg. [part. pres. di accogliere, p...",accogliente /ak:o'ʎɛnte/ agg. [part. pres. di ...,0.0


In [39]:
df_samples.to_csv('df_treccani.csv', index=False, sep=";")
df_samples.head()

,parola,parola2,w,d0,s0,no_match
0,A CALCIO,A CALCIO,a calcio,,,0.0
1,A PALLAVOLO,A PALLAVOLO,a pallavolo,,,0.0
2,A SCUOLA,A SCUOLA,a scuola,,,0.0
3,ABBINARE I COLORI,ABBINARE I COLORI,abbinare i colori,,,0.0
4,ACCOGLIENTE,ACCOGLIENTE,accogliente,"accogliènte agg. [part. pres. di accogliere, p...",accogliente /ak:o'ʎɛnte/ agg. [part. pres. di ...,0.0


In [40]:
# # GENERAL APPEARANCE
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)
# mestieri = df_samples[df_samples.s0.str.contains('mest.', regex=False,na=False)].shape[0]
# professioni = df_samples[df_samples.s0.str.contains('prof.', regex=False,na=False)].shape[0]
# print(f"Numero di mestieri : {mestieri}\nNumero di professioni : {professioni}")

In [41]:
#CICLO CONVERSIONE GENERE (-ESSA/-TRICE)
df_samples["parola_genere1"] = df_samples.parola2

# for i in range(len(df_samples.parola2)):
#     if df_samples.parola2[i] == "ARCHITETTRICE": #caso particolare di mispelling
#         print(f"Prima: {df_samples.parola2[i]}")
#         df_samples["parola_genere1"][i] = df_samples["parola_genere1"][i].replace("ARCHITETTRICE", "ARCHITETTA")
#         print(f"Dopo: {df_samples['parola_genere1'][i]}")

for i in range(len(df_samples.parola2)):
    if df_samples.parola2[i][-5:] == "TRICE":
        print(f"Prima: {df_samples.parola2[i]}")
        df_samples["parola_genere1"][i] = df_samples["parola_genere1"][i].replace("TRICE", "TORE")
        print(f"Dopo: {df_samples['parola_genere1'][i]}")

print(df_samples['parola_genere1'])


Prima: ATTRICE
Dopo: ATTORE
Prima: DIRETTRICE
Dopo: DIRETTORE
Prima: MERETRICE
Dopo: MERETORE
Prima: OPERATRICE
Dopo: OPERATORE
Prima: PULITRICE
Dopo: PULITORE
Prima: SPILLATRICE
Dopo: SPILLATORE
Prima: STIRATRICE
Dopo: STIRATORE
0               A CALCIO
1            A PALLAVOLO
2               A SCUOLA
3      ABBINARE I COLORI
4            ACCOGLIENTE
             ...        
735               VOYEUR
736         WEBMARKETING
737                  ZIA
738                  ZIO
739              ZOCCOLA
Name: parola_genere1, Length: 740, dtype: object


C:\Users\Nauel\AppData\Local\Temp\ipykernel_10676\55469948.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_samples["parola_genere1"][i] = df_samples["parola_genere1"][i].replace("TRICE", "TORE")


In [42]:
for i in range(len(df_samples.parola2)):
    if df_samples.parola2[i] == "COMMESSA": # commesso è un caso particolare che può essere affrontato in modo a sè stante
        df_samples["parola_genere1"][i] = df_samples["parola_genere1"][i].replace("COMMESSA", "COMMESSO")
    if df_samples.parola2[i][-4:] == "ESSA":
        print(df_samples["parola_genere1"][i])
        df_samples["parola_genere1"][i] = df_samples["parola_genere1"][i].replace("ESSA", "E")
        print(df_samples["parola_genere1"][i])

corpus = list(df_samples.parola_genere1)

In [43]:
df1 = df_samples.sort_values('parola_genere1').copy()

In [44]:
# PULIZIA PER DISTANZA DI LEVESHTEIN 
words_to_modify = []


for i in range(0, len(df1["parola_genere1"])):
    for j in range(i+1, len(df1["parola_genere1"])):   # confronto ogni coppia di parole
        w1 = math.trunc(len(df1["parola_genere1"][i])*0.99)
        w2 = math.trunc(len(df1["parola_genere1"][j])*0.99)
        if lev(df1["parola_genere1"][i][:w1], df1["parola_genere1"][j][:w2]) == 0:    # se l'N% (in questo caso 99%, arrotondato sempre per difetto) delle due parole a partire dall'inzio è uguale
            if lev(df1["parola_genere1"][i][w1:], df1["parola_genere1"][j][w2:]) > 0: # allora controlla che il restante 1-N% sia differente, appendi solo quelli che hanno una differenza (quindi non parole uguali) 
                words_to_modify.append([df1["parola_genere1"][i], df1["parola_genere1"][j]])

lenght = len(words_to_modify)
print(words_to_modify, f"\nLenght of the list: {lenght}")
dict(words_to_modify)

df1["parola_genere1"] = list(map(lambda x: dict(words_to_modify).get(x, x), df1["parola_genere1"]))
df1["parola_genere1"]


[['BALLERINA', 'BALLERINO'], ['BELLA', 'BELLO'], ['CAMERIERA', 'CAMERIERE'], ['CASALINGA', 'CASALINGO'], ['COGNATA', 'COGNATO'], ['COMICA', 'COMICO'], ['FIGA', 'FIGO'], ['FIGLIA', 'FIGLIO'], ['GIARDINIERA', 'GIARDINIERE'], ['IDRAULICA', 'IDRAULICO'], ['LAVANDAIA', 'LAVANDAIO'], ['MAESTRA', 'MAESTRO'], ['MASSAIA', 'MASSAIO'], ['MECCANICA', 'MECCANICO'], ['NONNA', 'NONNO'], ['OSTETRICA', 'OSTETRICO'], ['RAGAZZA', 'RAGAZZO'], ['SARTA', 'SARTO'], ['SIGNORA', 'SIGNORE'], ['SPOSA', 'SPOSO'], ['SUOCERA', 'SUOCERO'], ['ZIA', 'ZIO']] 
Lenght of the list: 22


0               A CALCIO
1            A PALLAVOLO
2               A SCUOLA
3      ABBINARE I COLORI
4            ACCOGLIENTE
             ...        
736         WEBMARKETING
737                  ZIO
738                  ZIO
739              ZOCCOLA
516                 papà
Name: parola_genere1, Length: 740, dtype: object

In [45]:
#CREAZIONE NUOVO DATAFRAME CON CORREZIONI MISMATCHED DEFINITIONS
# dff = pd.DataFrame()

# definizioni = dict(zip(df1.parola2, df1.d0))
# sinonimi = dict(zip(df1.parola2, df1.s0))

# dff['parola_origine'] = df1['parola']
# dff['parola_origine1'] = df1['parola2']
# dff['parola_genere_unico'] = df1['parola_genere1']
# dff['definizione'] = df1['parola_genere1']
# dff['sinonimi'] = df1['parola_genere1']

# dff['definizione'] = dff['definizione'].map(definizioni)
# dff['sinonimi'] = dff['sinonimi'].map(sinonimi)
# dff.head()

In [46]:
#CHECKPOINT
output_for_comparison = dff[['parola_origine','parola_genere_unico']]
output_for_comparison.to_csv('Risultati conversione genere.csv', index=False, sep = ';')
output_for_comparison.head()

NameError: name 'dff' is not defined

In [48]:
dff = pd.read_csv('Risultati conversione genere.csv', sep = ';')
df1.head()

,parola,parola2,w,d0,s0,no_match,parola_genere1
0,A CALCIO,A CALCIO,a calcio,,,0.0,A CALCIO
1,A PALLAVOLO,A PALLAVOLO,a pallavolo,,,0.0,A PALLAVOLO
2,A SCUOLA,A SCUOLA,a scuola,,,0.0,A SCUOLA
3,ABBINARE I COLORI,ABBINARE I COLORI,abbinare i colori,,,0.0,ABBINARE I COLORI
4,ACCOGLIENTE,ACCOGLIENTE,accogliente,"accogliènte agg. [part. pres. di accogliere, p...",accogliente /ak:o'ʎɛnte/ agg. [part. pres. di ...,0.0,ACCOGLIENTE


In [49]:
# AGGIUNTA DI PAROLA CHIAVE MESTIERE/PROFESSIONE 
# dff['parola_con_descrizione'] = np.where(dff.sinonimi.str.contains('mest.', regex=False), 
#                                                 dff['parola_genere_unico'] + " LAVORO",
#                                                 dff['parola_genere_unico'])

# dff['parola_con_descrizione'] = np.where(dff.sinonimi.str.contains('prof.', regex=False), 
#                                                 dff['parola_genere_unico'] + " PROFESSIONE",
#                                                 dff['parola_con_descrizione'])

# corpus = dff['parola_con_descrizione'].values.tolist()
# corpus

In [50]:
# STATISTICHE GENERALI
total_words = df_samples.shape[0]
unique_words = df_samples['parola'].nunique()
unique_words_treccani = df_samples['parola2'].nunique()
unique_words_gender= dff['parola_genere_unico'].nunique()

print(f"Osservazioni totali: {total_words}\
        \nOsservazioni uniche: {total_words}\
        \nOsservazioni uniche (treccani): {unique_words_treccani}\
        \nOsservazioni uniche (genere): {unique_words_gender}")

# items = Counter(corpus).keys()
# print("Osservazioni uniche (leveshtein):", len(items))

Osservazioni totali: 740        
Osservazioni uniche: 740        
Osservazioni uniche (treccani): 643        
Osservazioni uniche (genere): 627


In [51]:
# CORPUS CON FEMMINILI/MASCHILI UNITI
corpus = list(map(lambda x: dict(words_to_modify).get(x, x), corpus))
corpus

items = Counter(corpus).keys()
print("Osservazioni uniche (leveshtein):", len(items))

Osservazioni uniche (leveshtein): 617


In [52]:
# # SEPARAZIONE CORPUS PAROLE SINGOLE DA FRASI (NON NECESSARIO, SI PUO LAVORARE ANCHE CON IL CORPUS PER INTERO)
# corpus_single_word_eng = []
# i = 0

# for item in corpus:
#     if (len(item.split()))<=1:
#         corpus_single_word_eng.append(corpus[i])
#     i = i+1
# corpus_single_word_eng

# corpus_multiple_word_eng = []
# i = 0

# for item in corpus:
#     if (len(item.split()))>1:
#         corpus_multiple_word_eng.append(corpus[i])
#     i = i+1
# len(corpus_multiple_word_eng)

In [54]:
# TRADUZIONE DELLA LISTA DELLE ESPRESSIONI
from deep_translator import GoogleTranslator

eng_corpus = []
for i in range(len(corpus)):
    translated = GoogleTranslator(source='italian', target='english').translate(corpus[i])
    eng_corpus.append(translated)
    
[x.upper() for x in eng_corpus]
eng_corpus

['FOOTBALL',
 'AT VOLLEYBALL',
 'AT SCHOOL',
 'MATCH COLORS',
 'WELCOMING',
 'COMPANION',
 'CARING FOR THE CHILDREN',
 'DOG TRAINER',
 'EASY GOING',
 'BUSY',
 'AGENT',
 'REAL ESTATE AGENT',
 'ADJUST',
 'ADJUST THE ELECTRICAL SYSTEM',
 'ADJUSTER',
 'ADJUSTER',
 'ADJUSTER',
 'FARMER',
 'SUPPORT WORKER',
 'DOMESTIC HELP',
 'HELP',
 'AT THE PC',
 'ALLY',
 'TRAINER',
 'STUDENT',
 'STUDENT',
 'ALPHA extension',
 'MOUNTAINEEPER',
 'ATHLETE',
 'STUDENT',
 'STUDENT',
 'LOVER',
 'FRIEND',
 'FRIEND',
 'TRAINER OF PAROKETS',
 'ADMINISTRATOR',
 'ADMINISTRATOR',
 'HOUSE ADMINISTRATOR',
 'CONDOMINIUM ADMINISTRATOR',
 'ANALYST',
 'ANESTHETIST',
 'ANIMALIST',
 'ANIMATOR',
 'BEEKEEPER',
 'BENEFITER',
 'REFEREE',
 'ARCHAEOLOGIST',
 'ARCHAEOLOGIST',
 'ARCHITECT',
 'ARCHITECT',
 'FURNISH HOME',
 'FURNISHER',
 'FURNISHER',
 'GRINDER',
 'ARTISAN',
 'ARTISAN',
 'ARTIST',
 'LISTEN',
 'LISTENER',
 'LISTENER',
 'COUNCILOR',
 'COUNCILOR',
 'INSURER',
 'ASSISTANT',
 'SOCIAL WORKER',
 'ASTROPHYSICS',
 'ASTRONAUT',


In [58]:
eng_corpus_checkpoint = pd.DataFrame(eng_corpus, columns=['Expressions'])
eng_corpus_checkpoint.to_csv(f"{PATH_CHECKPOINTS}\lista_parole_tradotte.csv", index = False)

In [24]:
# eng_corpus_checkpoint = pd.DataFrame(eng_corpus, columns=['Expressions'])
eng_corpus_checkpoint.to_pickle(f"{PATH_CHECKPOINTS}\corpus_inglese_grezzo.pkl")

In [25]:
eng_corpus_checkpoint = pd.read_pickle(f"{PATH_CHECKPOINTS}\corpus_inglese_grezzo.pkl")

eng_corpus_checkpoint.head()

0         FOOTBALL
1    AT VOLLEYBALL
2        AT SCHOOL
3     MATCH COLORS
4        WELCOMING
Name: Expressions, dtype: object

In [26]:
eng_corpus_checkpoint['max_words'] = eng_corpus_checkpoint.apply(lambda x: len(x.split()))

In [27]:
df_splitted = eng_corpus_checkpoint['Expressions'].str.split(expand=True)
df_splitted.fillna('', inplace = True)
df_splitted = df_splitted.apply(lambda x: x.str.upper() if x.dtype == "object" else x)

KeyError: 'Expressions'

In [ ]:
result_dfs = []
for j in range(df_splitted.shape[1]):
    words = df_splitted[j].tolist()
    result = []
    for i in range(len(words)):
        a = wn.synsets(words[i], lang = 'eng')   
        if len(a) == 0:
            result.append([words[i], [], [], [], []])
        else:
            definitions, synonyms, hyponyms, hypernyms = [], [], [], []
            for synset in a:
                definitions.append(synset.definition())
                synonyms.extend([syn.name() for syn in synset.lemmas()])
                hyponyms.extend([hypo.lemmas()[0].name() for hypo in synset.hyponyms()])
                hypernyms.extend([hyper.lemmas()[0].name() for hyper in synset.hypernyms()])
            result.append([words[i], definitions, synonyms, hyponyms, hypernyms])
    result_df = pd.DataFrame(result, columns=[f'word{j}', f'definitions{j}', f'synonyms{j}', f'hyponyms{j}', f'hypernyms{j}'])
    result_dfs.append(result_df)

final_df = pd.concat(result_dfs, axis=1)
final_df.columns = pd.MultiIndex.from_tuples([(f'word{i//5}', f'{col}') if i % 5 == 0 else (f'word{i//5}', f'{col}') for i, col in enumerate(final_df.columns)])


In [ ]:
pd.set_option("display.max_rows", 20, "display.max_columns", 20)
# final_df = final_df.replace(to_replace=r"\[",value="\(", regex=True)
# final_df = final_df.replace(to_replace=r"\]",value="\(", regex=True)
final_df = final_df.applymap(lambda x: str(x).replace('[', '(').replace(']', ')'))
final_df.head()

word0                                                     \
       word0                                       definitions0   
0   FOOTBALL  ("any of various games played with a ball (rou...   
1         AT  ('a highly unstable radioactive element (the h...   
2         AT  ('a highly unstable radioactive element (the h...   
3      MATCH  ('lighter consisting of a thin piece of wood o...   
4  WELCOMING  ('accept gladly', 'bid welcome to; greet upon ...   

                                                      \
                                           synonyms0   
0          ('football', 'football_game', 'football')   
1       ('astatine', 'At', 'atomic_number_85', 'at')   
2       ('astatine', 'At', 'atomic_number_85', 'at')   
3  ('match', 'lucifer', 'friction_match', 'match'...   
4  ('welcome', 'welcome', 'receive', 'welcome', '...   

                                                      \
                                           hyponyms0   
0  ('American_football', 'professional_football',...   
1                                                 ()   
2                                                 ()   
3  ('fusee', 'kitchen_match', 'safety_match', 'sl...   
4                                                 ()   

                                                           word1  \
                                          hypernyms0       word1   
0            ('contact_sport', 'field_game', 'ball')               
1  ('chemical_element', 'halogen', 'Laotian_monet...  VOLLEYBALL   
2  ('chemical_element', 'halogen', 'Laotian_monet...      SCHOOL   
3  ('lighter', 'contest', 'lighter', 'duplicate',...      COLORS   
4                     ('accept', 'greet', 'receive')               

                                                      \
                                        definitions1   
0                                                 ()   
1  ('a game in which two teams hit an inflated ba...   
2  ('an educational institution', 'a building whe...   
3  ('a flag that shows its nationality', 'a disti...   
4                                                 ()   

                                                      \
                                           synonyms1   
0                                                 ()   
1    ('volleyball', 'volleyball_game', 'volleyball')   
2  ('school', 'school', 'schoolhouse', 'school', ...   
3  ('colors', 'colours', 'colors', 'colours', 'co...   
4                                                 ()   

                                                      \
                                           hyponyms1   
0                                                 ()   
1                                                 ()   
2  ('academy', 'alma_mater', 'conservatory', 'cor...   
3  ('ensign', 'achromatic_color', 'chromatic_colo...   
4                                                 ()   

                                                      ... word5               \
                                          hypernyms1  ... word5 definitions5   
0                                                 ()  ...                 ()   
1                             ('court_game', 'ball')  ...                 ()   
2  ('educational_institution', 'building', 'educa...  ...                 ()   
3  ('flag', 'emblem', 'visual_property', 'interes...  ...                 ()   
4                                                 ()  ...                 ()   

                                 word6                                   \
  synonyms5 hyponyms5 hypernyms5 word6 definitions6 synonyms6 hyponyms6   
0        ()        ()         ()                 ()        ()        ()   
1        ()        ()         ()                 ()        ()        ()   
2        ()        ()         ()                 ()        ()        ()   
3        ()        ()         ()                 ()        ()        ()   
4        ()        ()         ()                 ()        ()        ()   

        

In [ ]:
result[0] #first results of the first word
result[0][1] #all definitions of the first word
result[0][1][0] #first definitions of the first word

NameError: name 'result' is not defined

In [28]:
full_text = final_df.apply(lambda x: ' '.join(x.astype(str)), axis=1)

In [29]:
full_text

0      FOOTBALL ("any of various games played with a ...
1      AT ('a highly unstable radioactive element (th...
2      AT ('a highly unstable radioactive element (th...
3      MATCH ('lighter consisting of a thin piece of ...
4      WELCOMING ('accept gladly', 'bid welcome to; g...
                             ...                        
735    VOYEUR ('a viewer who enjoys seeing the sex ac...
736    WEB ('an intricate network suggesting somethin...
737    UNCLE ('the brother of your father or mother; ...
738    UNCLE ('the brother of your father or mother; ...
739    SLUT ('a dirty untidy woman', 'a woman adulter...
Length: 740, dtype: object

In [31]:
full_text.to_pickle(f"{PATH_CHECKPOINTS}\Full_text.pkl")

In [64]:
# CLUSTERING SENZA DEFINIZIONI
corpus_embeddings = model.encode(full_text)

# Perform kmean clustering
groups = [5, 8, 10, 15, 20, 80, 100, 125, 150, 175, 200, 250, 300]

for num_clusters in groups:
    clustering_model = KMeans(n_clusters=num_clusters)
    clustering_model.fit(corpus_embeddings)
    cluster_assignment = clustering_model.labels_

    clustered_sentences = [[] for i in range(num_clusters)]
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        clustered_sentences[cluster_id].append(full_text[sentence_id])

    with open(f'C:/Users/Nauel/Desktop/Lavoro UNI/Cluster_loop/Clusters con {num_clusters} gruppi & definizioni.txt', "w") as f:
        print(f"Clustering with {num_clusters} groups:", file=f)
        for i, cluster in enumerate(clustered_sentences):
            print("Cluster ", i+1, file=f)
            print(cluster, file=f)
            print("", file=f)
            

In [252]:
# CLUSTERING CON DEFINIZIONI (QUELLO CHE C'E' DOPO NON SERVE)
corpus_embeddings = model.encode(full_text)

# Perform K-MEAN clustering
num_clusters = 15
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(full_text[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")


Cluster  1
['FOOTBALL; any of various games played with a ball (round or oval) in which two teams try to kick or carry or propel the ball into each other s goal;   ;   football ,  football_game  ', 'WELCOMING; accept gladly;   I welcome your proposals  ;   welcome  ', 'BUSY; keep busy with;   She busies herself with her butterfly collection  ;   busy ,  occupy  ', 'ADJUST; alter or regulate so as to achieve accuracy or conform to a standard;   Adjust the clock, please ,  correct the alignment of the front wheels  ;   adjust ,  set ,  correct  ', 'HELP; the activity of contributing to the fulfillment of a need or furtherance of an effort or purpose;   he gave me an assist with the housework ,  could not walk without assistance ,  rescue party went to their aid ,  offered his help in unloading  ;   aid ,  assist ,  assistance ,  help  ', 'ALLY; a friendly nation;   ;   ally  ', 'ASTROPHYSICS; the branch of astronomy concerned with the physical and chemical properties of celestial bodies;

In [107]:
import re

# Open the text file
for num in groups:
    with open(f'C:/Users/Nauel/Desktop/Lavoro UNI/cluster_loop_clean/definitions/Clusters con {num} gruppi & definizioni.txt', 'r') as file:
        # Read the contents of the file
        text = file.read()
        
    # Use the re.sub() function to replace all instances of text inside parentheses with an empty string
    text = re.sub(r'\([^()]*\)', '', text)
    text = re.sub(r'\([^()]*\)', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'(Cluster \d+)', '\n\\1\n', text)

    # Write the modified text back to the file
    with open(f'C:/Users/Nauel/Desktop/Lavoro UNI/cluster_loop_clean/definitions/Clusters con {num} gruppi & definizioni.txt', 'w') as file:
        file.write(text)

In [130]:
# CLUSTERIZZAZIONE CON DATAFRAME TOTALE (K-MEAN)
# Corpus with example sentences
corpus_embeddings = model.encode(complete_df['merged'])

# Perform K-MEAN clustering
num_clusters = 90
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(complete_df['merged'][sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  1
['DOMESTIC HELP', 'DOMESTIC WORKER', 'IN THE HOUSEHOLD WORK']

Cluster  2
['WAITRESS; a woman waiter;   ;   waitress  ', 'WIFE; a married woman a man s partner in marriage;   ;   wife ,  married_woman  ', 'FIANCE; a man who is engaged to be married;   ;   fiance ,  groomtobe  ', 'HUSBAND; a married man a woman s partner in marriage;   ;   husband ,  hubby ,  married_man  ', 'WIFE; a married woman a man s partner in marriage;   ;   wife ,  married_woman  ', 'WIFE; a married woman a man s partner in marriage;   ;   wife ,  married_woman  ', 'MAN; an adult person who is male (as opposed to a woman);   there were two women and six men on the bus  ;   man ,  adult_male  ', 'WIFE; a married woman a man s partner in marriage;   ;   wife ,  married_woman  ', 'GROOM; a man participant in his own marriage ceremony;   ;   groom ,  bridegroom  ', 'MAN; an adult person who is male (as opposed to a woman);   there were two women and six men on the bus  ;   man ,  adult_male  ']

Cluster  

In [27]:
# AGGLOMERATIVE CLUSTERING (NON PERFORMA BENE)
corpus_embeddings = model.encode(dataframe['merged'])

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Perform kmean clustering
clustering_model = AgglomerativeClustering(n_clusters=None, affinity = 'cosine', linkage='single', distance_threshold=0.2)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []

    clustered_sentences[cluster_id].append(dataframe['merged'][sentence_id])

for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  32
['ACCOGLIENTE; affording pleasure being in harmony with your taste or likings;   we had a pleasant evening together ,  a pleasant scene ,  pleasant sensations  ;   pleasant  ', 'DOLCE; affording pleasure being in harmony with your taste or likings;   we had a pleasant evening together ,  a pleasant scene ,  pleasant sensations  ;   pleasant  ']

Cluster  390
['ACCOMPAGNATRICE; an attendant who is employed to accompany someone;   ;   escort  ']

Cluster  239
['AFFABILE; feeling or showing love and affection;   loving parents ,  loving glances  ;   loving  ']

Cluster  1
['AFFACCENDATA; ; ; ', 'AGGIUSTATORE; ; ; ', 'AGGIUSTATRICE; ; ; ', 'AGGIUSTATUTTO; ; ; ', 'ALLIEVA; ; ; ', 'ALTLETA; ; ; ', 'ANIMALISTA; ; ; ', 'ANIMATORE; ; ; ', 'APPROFITTATORE; ; ; ', 'ARREDATRICE; ; ; ', 'ARTIGIANA; ; ; ', 'ASSESSORA; ; ; ', 'ASSICURATRICE; ; ; ', 'BADANTE; ; ; ', 'BAGNINA; ; ; ', 'BANCARIA; ; ; ', 'BANCARIO; ; ; ', 'BARZELLETTIERE; ; ; ', 'BENZINAIA; ; ; ', 'BENZINAIO; ; ; ', 'BIDELLA; 

In [ ]:
# ALTRO TIPO DI CLUSTERING (NON PERFORMA BENE)
corpus_embeddings = model.encode(corpus)

# Perform K-MEAN clustering
clustering_model = DBSCAN(eps= 1.6, min_samples= 5, metric='euclidean', 
                          metric_params= None, algorithm= 'auto',
                          leaf_size= 5 )
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

In [60]:
b = pd.read_csv(f"{PATH_CHECKPOINTS}\man, woman, person.csv")

In [64]:
pd.set_option("display.max_rows", None)
b.head(200)



,Unnamed: 0,ita,list_of_words,MAN,WOMAN,PERSON,associated_word
0,0,A CALCIO,FOOTBALL,0.091239,0.140247,0.133595,woman
1,1,A PALLAVOLO,AT VOLLEYBALL,0.052370,0.134272,0.132421,woman
2,2,A SCUOLA,AT SCHOOL,-0.055784,0.069705,0.081704,person
3,3,ABBINARE I COLORI,MATCH COLORS,0.018944,0.106417,0.129342,person
4,4,ACCOGLIENTE,WELCOMING,-0.100408,0.055272,0.065517,person
5,5,ACCOMPAGNATRICE,COMPANION,-0.000364,0.094498,0.164992,person
6,6,ACCUDIRE I FIGLI,CARING FOR THE CHILDREN,-0.037303,0.091581,0.100823,person
7,7,ADDESTRATORE DI CANI,DOG TRAINER,0.042607,0.066978,0.108020,person
8,8,AFFABILE,EASY GOING,0.025456,0.092467,0.095966,person
9,9,AFFACCENDATA,BUSY,-0.039940,0.010093,0.090365,person
